In [16]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import numpy as np
import pandas as pd
from pathlib import Path
pd.set_option('display.max_rows', 25990)
pd.set_option('display.max_columns', 200)

In [21]:
# Load datasets
df_ap  = pd.read_csv("/content/drive/MyDrive/CP_UMBC/ Feature Engineering/AP_Features/ap_student_agg_without_DL.csv")


In [22]:
df_ap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25926 entries, 0 to 25925
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   EmployeeID                 25926 non-null  int64  
 1   AP_num_tests               25926 non-null  int64  
 2   AP_unique_codes            25926 non-null  int64  
 3   AP_any_credit              25926 non-null  int64  
 4   AP_total_transfer_credits  25926 non-null  float64
 5   AP_avg_score               25926 non-null  float64
 6   AP_max_score               25926 non-null  float64
 7   AP_ct_art                  25926 non-null  float64
 8   AP_ct_computer             25926 non-null  float64
 9   AP_ct_english              25926 non-null  float64
 10  AP_ct_language             25926 non-null  float64
 11  AP_ct_math                 25926 non-null  float64
 12  AP_ct_social               25926 non-null  float64
 13  AP_ct_stem                 25926 non-null  flo

In [23]:
df_aid = pd.read_csv("/content/drive/MyDrive/CP_UMBC/ Feature Engineering/Aid_Features/aid_second.csv")

/tmp/ipython-input-3913245619.py:1: DtypeWarning: Columns (22,23,24,25,26,27,43,44,45,46,47,48,104,106) have mixed types. Specify dtype option on import or set low_memory=False.
  df_aid = pd.read_csv("/content/drive/MyDrive/CP_UMBC/ Feature Engineering/Aid_Features/aid_second.csv")


In [24]:
df_aid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25960 entries, 0 to 25959
Columns: 135 entries, StudentKey to SupportBin
dtypes: float64(55), int64(18), object(62)
memory usage: 26.7+ MB


In [25]:
#  normalize EmployeeID in both datasets before merging

df_ap["EmployeeID"]  = df_ap["EmployeeID"].astype(str).str.strip()
df_aid["EmployeeID"] = df_aid["EmployeeID"].astype(str).str.strip()

# Perform LEFT JOIN from df_aid → add AP features
merged = pd.merge(
    df_aid,
    df_ap,
    on="EmployeeID",
    how="left"
)

print(" Merge complete")
print("Merged shape:", merged.shape)

# How many matched
matches = merged["AP_num_tests"].notna().sum()
print("Students with AP data matched:", matches)
print("Match rate (%):", round(100 * matches / len(df_aid), 2))

# Quick sanity check
#merged[["EmployeeID","Graduated","AP_num_tests","AP_total_transfer_credits","AP_STEM_ratio"]].head(10)


 Merge complete
Merged shape: (25960, 157)
Students with AP data matched: 25960
Match rate (%): 100.0


In [26]:
# Fill missing values by type (numeric=0, categorical="Unknown")



# Work on a copy so we can compare later if needed
merged_filled = merged.copy()

# Identify column types
num_cols = merged_filled.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = merged_filled.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"Numeric cols: {len(num_cols)} | Categorical cols: {len(cat_cols)}")

#  Missing counts BEFORE
na_num_before = merged_filled[num_cols].isna().sum().sum()
na_cat_before = merged_filled[cat_cols].isna().sum().sum()
print(f"NaNs BEFORE  -> numeric: {na_num_before:,} | categorical: {na_cat_before:,}")

#  Fill
merged_filled[num_cols] = merged_filled[num_cols].fillna(0)
merged_filled[cat_cols] = merged_filled[cat_cols].fillna("Unknown")

#  Missing counts AFTER
na_num_after = merged_filled[num_cols].isna().sum().sum()
na_cat_after = merged_filled[cat_cols].isna().sum().sum()
print(f"NaNs AFTER   -> numeric: {na_num_after:,} | categorical: {na_cat_after:,}")



Numeric cols: 94 | Categorical cols: 63
NaNs BEFORE  -> numeric: 442,232 | categorical: 687,829
NaNs AFTER   -> numeric: 0 | categorical: 0


In [27]:
# Duplicate checks

#  full-row duplicates (entire rows repeated)
full_dupes = merged_filled.duplicated().sum()
print(f"Full-row duplicate count: {full_dupes}")

#  duplicates based only on EmployeeID (the key)
id_dupes = merged_filled["EmployeeID"].duplicated().sum()
dupe_ids = merged_filled.loc[merged_filled["EmployeeID"].duplicated(), "EmployeeID"].unique()
print(f"Duplicate EmployeeIDs: {id_dupes}")
if len(dupe_ids) > 0:
    print("\nExample duplicate EmployeeIDs:", dupe_ids[:10])
else:
    print("\n No duplicated EmployeeIDs found — each student appears only once.")

# sanity check of unique ID coverage
unique_ids = merged_filled["EmployeeID"].nunique()
print(f"\nUnique EmployeeIDs: {unique_ids}  |  Total rows: {len(merged_filled)}")
if unique_ids == len(merged_filled):
    print(" 1 row per student confirmed.")
else:
    print(" Some students appear multiple times.")


Full-row duplicate count: 0
Duplicate EmployeeIDs: 34

Example duplicate EmployeeIDs: ['3000331717' '4000016032' '4000121931' '4000200976' '4000213214'
 '4000328505' '4000223723' '4000408636' '4000339378' '4000296234']

Unique EmployeeIDs: 25926  |  Total rows: 25960
 Some students appear multiple times.


In [28]:
#  show all columns for duplicate students
import pandas as pd

# Get list of duplicate EmployeeIDs
dupe_ids = merged_filled.loc[merged_filled["EmployeeID"].duplicated(), "EmployeeID"].unique()
print(f"Total duplicate EmployeeIDs: {len(dupe_ids)}")

# Filter full rows for those duplicates
subset_dupes_full = merged_filled[merged_filled["EmployeeID"].isin(dupe_ids)].sort_values(
    ["EmployeeID", "MatricAcademicYear"]
)

print(f"\nTotal rows shown: {len(subset_dupes_full)} (across {len(dupe_ids)} students)")
print("Showing first few duplicate students (all columns):")

# Display first few duplicates fully
#display(subset_dupes_full.head(100))


Total duplicate EmployeeIDs: 12

Total rows shown: 46 (across 12 students)
Showing first few duplicate students (all columns):


In [29]:
# column-wise comparison for duplicate students

# Get duplicate EmployeeIDs
dupe_ids = merged_filled.loc[merged_filled["EmployeeID"].duplicated(), "EmployeeID"].unique()
print(f"Duplicate EmployeeIDs: {len(dupe_ids)}")

#  Create a function to find which columns differ for each duplicate student
def find_differences(df, emp_id):
    subset = df[df["EmployeeID"] == emp_id]
    # For each column, check if more than one unique non-null value exists
    diffs = {}
    for c in df.columns:
        vals = subset[c].dropna().unique()
        if len(vals) > 1:
            diffs[c] = vals
    return pd.Series({
        "EmployeeID": emp_id,
        "NumRows": subset.shape[0],
        "NumDiffCols": len(diffs),
        "DiffCols": list(diffs.keys()),
    })

#  Apply to all duplicate IDs
diff_summary = pd.concat([find_differences(merged_filled, i) for i in dupe_ids], axis=1).T

#print("Summary of what differs per duplicate student:")
#display(diff_summary.head(20))

#  Pick one example student with multiple differing columns to inspect fully
#example_id = diff_summary.sort_values("NumDiffCols", ascending=False)["EmployeeID"].iloc[0]
#print(f"\nInspecting EmployeeID: {example_id}")
#display(merged_filled[merged_filled["EmployeeID"] == example_id])


Duplicate EmployeeIDs: 12


In [30]:
# collapse duplicates safely to one row per student


def custom_agg(series):
    """Aggregation logic depending on data type."""
    # For object/string columns -> join unique non-null values
    if series.dtype == "O" or series.dtype == "object":
        uniq = series.dropna().unique()
        return ", ".join(map(str, uniq)) if len(uniq) > 0 else np.nan
    # For boolean / int / float -> take maximum (safe for indicators & amounts)
    else:
        return series.max(skipna=True)

# Apply groupby + custom aggregation
merged_grouped = merged_filled.groupby("EmployeeID", as_index=False).agg(custom_agg)

# Validation summary
print("Deduplication complete")
print(f"Rows after collapse: {merged_grouped.shape[0]:,}")
print(f"Unique EmployeeIDs : {merged_grouped['EmployeeID'].nunique():,}")
print(f"Remaining duplicates: {merged_grouped['EmployeeID'].duplicated().sum()}")

# Check a few of the previously duplicated IDs to confirm merge worked
#dupe_ids = ['3000331717', '4000408636', '4000223723']
#display(merged_grouped[merged_grouped['EmployeeID'].isin(dupe_ids)])


Deduplication complete
Rows after collapse: 25,926
Unique EmployeeIDs : 25,926
Remaining duplicates: 0


/tmp/ipython-input-3056071280.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_grouped = merged_filled.groupby("EmployeeID", as_index=False).agg(custom_agg)


In [31]:
from pathlib import Path

# Define output directory and file paths
SAVE_DIR = Path("/content/drive/MyDrive/CP_UMBC/ Feature Engineering/Merged")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

csv_path = SAVE_DIR / "merged_clean_dedup.csv"
parquet_path = SAVE_DIR / "merged_clean_dedup.parquet"

# Save both formats
merged_grouped.to_csv(csv_path, index=False)
merged_grouped.to_parquet(parquet_path, index=False)

print(" Clean merged dataset saved successfully.")
print(f"CSV path    : {csv_path}")
print(f"Parquet path: {parquet_path}")
print(f"Shape       : {merged_grouped.shape}")


 Clean merged dataset saved successfully.
CSV path    : /content/drive/MyDrive/CP_UMBC/Feature_Engneering/Merged/merged_clean_dedup.csv
Parquet path: /content/drive/MyDrive/CP_UMBC/Feature_Engneering/Merged/merged_clean_dedup.parquet
Shape       : (25926, 157)
